In [1]:
import time
import random
import numpy as np

import messages
from simulator import Simulator
from agent import Agent
from unity_env import UnityEnv

<class 'sac.sac_agent.SACAgent'>
<class 'sac.sac_env.SACEnv'>


In [2]:
simulator = Simulator(port=5004)
#env = UnityEnv(simulator, 9, 2, 1, lambda: np.random.random((2,)))
#agent = Agent(env)
#buffer = ReplayBuffer(17, 2, int(1e6))

In [3]:
simulator.start_server()
serv = simulator.server

In [225]:
import json
from messages.controll_message import ControllMessage
class SingleFieldMessage:
    def __init__(self, x) -> None:
        self.value = x
    def to_json(self):
        return json.dumps(self.__dict__)
    

msg = messages.RequestMessage("test", "").to_json()
msg2 = messages.RequestMessage("timeScale", SingleFieldMessage(2.).to_json()).to_json()
msg3 = messages.RequestMessage("pause", SingleFieldMessage(False).to_json()).to_json()
msg4 = messages.RequestMessage("reset", "").to_json()
msg5 = messages.RequestMessage("step", ControllMessage(5, 5).to_json()).to_json()


In [56]:
serv.clients[-1].socket.sendall(msg.encode())

In [67]:
serv.clients[-1].socket.sendall(msg2.encode())

In [58]:
serv.clients[-1].socket.sendall(msg3.encode())

In [295]:
serv.clients[-1].socket.sendall(msg4.encode())

In [291]:
for i in range(100):
    serv.clients[-1].socket.sendall(msg5.encode())
    time.sleep(0.02)

In [ ]:
simulator.server.clients[-1].socket.sendall(b"foo")

In [ ]:
control_message = messages.ControllMessage(0, 10)
message = messages.RequestMessage("step", control_message.to_json())

simulator.server.send([message.to_json()])

response = messages.ResponseMessage.from_json(simulator.server.recive()[0])
observation_message = messages.ObservationMessage.from_json(response.value)
print(observation_message.to_json())


In [296]:
simulator.server.clients[-1].receive()

b'{"value":"{\\"agentPostion\\":{\\"x\\":8.245100021362305,\\"y\\":0.30000001192092898,\\"z\\":5.982548713684082},\\"agentRotation\\":{\\"x\\":0.0,\\"y\\":88.83110809326172,\\"z\\":0.0},\\"velocity\\":{\\"x\\":0.0,\\"y\\":0.0,\\"z\\":0.0},\\"angularVelocity\\":{\\"x\\":0.0,\\"y\\":0.0,\\"z\\":0.0},\\"redBallPosition\\":{\\"x\\":-1.1663446426391602,\\"y\\":0.25,\\"z\\":6.204244613647461},\\"blueBallPosition\\":{\\"x\\":0.9334774613380432,\\"y\\":-10.0,\\"z\\":2.549999952316284},\\"greenBallPosition\\":{\\"x\\":0.9334774613380432,\\"y\\":-10.0,\\"z\\":-4.090000152587891},\\"grayAreaPosition\\":{\\"x\\":-1.215153455734253,\\"y\\":0.009999999776482582,\\"z\\":7.588531494140625},\\"orangeAreaPosition\\":{\\"x\\":5.53000020980835,\\"y\\":-10.0,\\"z\\":3.6500000953674318},\\"whiteAreaPosition\\":{\\"x\\":-6.53000020980835,\\"y\\":-10.0,\\"z\\":-0.3357459604740143},\\"terminate\\":false,\\"reward\\":0.0}"}'

In [ ]:
simulator.server.clients[-1].socket.sendall(b"foo")

In [ ]:
agent.train()

In [ ]:
simulator.shutdown()